# **Import**

In [ ]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, plot_importance
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report, roc_curve
from sklearn.model_selection import cross_validate

# **23만 데이터**

In [ ]:
df = pd.read_csv('데이터경로')
df.shape
df['label'].value_counts()

# **입력 파일 정리**


In [ ]:
df_1 = df.drop(columns=['FILENAME', 'URL', 'Domain','Title','TLD'])
df_1

# **X, y값 지정**

In [ ]:
X = df_1.drop(columns=['label'])
X

In [ ]:
y = df_1['label'].values
y

# **데이터 분할**
학습 데이터셋 : 테스트 데이터셋 = 80: 20

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)


# **모델 파라미터 설정 및 loss값 error값 확인**
tree booster Parameter


In [ ]:
XGB = XGBClassifier(max_depth=5, reg_lambda=0.1, n_estimators=400, objective='binary:logistic')
evals = [(X_test, y_test)]
XGB.fit(X_train, y_train, early_stopping_rounds=100, eval_metric=["error","logloss"], eval_set=evals, verbose=True)

In [ ]:
y_train_pred = XGB.predict(X_train)
y_test_pred = XGB.predict(X_test)

# **테스트 평가지표**
성능평가
정확도, 오차 행렬, 정밀도, 재현율, F1스코어, ROC AUC

In [ ]:
#ROC AUC 계산을 위한 확률 값
y_test_pred_proba = XGB.predict_proba(X_test)[:, 1]

In [ ]:
test_accuracy = accuracy_score(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_pred_proba)

In [ ]:
print("Test Accuracy:", test_accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

In [ ]:
print("\n성능 보고서:\n", classification_report(y_test, y_test_pred))

# **시각화**
ROC Curve, 오차행렬, error, loglooss

In [ ]:
#ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_test_pred_proba)

plt.figure(figsize=(14,10))
plt.subplot(2, 2, 1)
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('XGB (ROC)')
plt.legend(loc='lower right')


#오차행렬
plt.subplot(2, 2, 2)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')


#에러
results = XGB.evals_result()

plt.subplot(2, 2, 3)
plt.plot(results['validation_0']['error'], label='Test Error')
plt.xlabel('Boosting Round')
plt.ylabel('Error')
plt.title('Test Error over Boosting Rounds')
plt.legend()


# Logloss per iteration
plt.subplot(2, 2, 4)
plt.plot(results['validation_0']['logloss'], label='Test Logloss')
plt.xlabel('Boosting Round')
plt.ylabel('Logloss')
plt.title('Test Logloss over Boosting Rounds')
plt.legend()


plt.tight_layout()
plt.show()

# **상위 특징 10개 추출**

In [ ]:
feature_importances = XGB.feature_importances_
top_features = np.argsort(feature_importances)[::-1][:10]
print("Top 10 Features", top_features)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 24))
plot_importance(XGB, ax=ax)

# **일반화 및 모델 성능 향상을 위한 하이퍼 파라미터 튜닝**

In [ ]:
#우선 교차검증부터 시작
start_time = time.time()
cross_val = cross_validate(
    XGB,
    X,
    y,
    cv=5,
    return_train_score=True,
    scoring='accuracy'
)
end_time = time.time()
total_cross_validation_time = end_time - start_time

#전체 교차 검증 시간 출력
total_cross_validation_time_minutes = total_cross_validation_time / 60
print("총 교차 검증 시간(분) :", total_cross_validation_time_minutes)
# 전체 교차 검증 점수 출력
print("훈련 점수:", np.mean(cross_val['train_score']))
print("검증 점수:", np.mean(cross_val['test_score']))

In [ ]:
from sklearn.model_selection import GridSearchCV
n_XGB = XGBClassifier(random_state=10)
param_grid = {
    'n_estimators': [1000, 2000, 3000],

}
grid_search = GridSearchCV(n_XGB, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)
best_params = grid_search.best_params_


In [ ]:
print("Best Parameters:", best_params)


In [ ]:
# XGBClasifier 초기화
param = {
    'n_estimators': 1000,
    'max_depth': None,
    'learning_rate': 0.3,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.5,
    'colsample_bytree': 0.55
    }
XGB_Best = xgb.XGBClassifier(**param)
# 모델 훈련 및 평가
eval_set = [(X_train, y_train), (X_test, y_test)]
XGB_Best.fit(X_train, y_train, eval_set=eval_set, eval_metric=["error", "logloss"], verbose=True)
# validation_0 = Train, validation_1 = Test

In [ ]:
# 에러 값, 손실 값, 최종 점수 확인
evals_result = XGB_Best.evals_result()
train_error = evals_result['validation_0']['error']
test_error = evals_result['validation_1']['error']
train_loss = evals_result['validation_0']['logloss']
test_loss = evals_result['validation_1']['logloss']
final_score = XGB_Best.score(X_test, y_test)
print("Train Error:", [f"{error:.4f}" for error in train_error])
print("Test Error:", [f"{error:.4f}" for error in test_error])
print("Train Loss:", [f"{loss:.4f}" for loss in train_loss])
print("Test Loss:", [f"{loss:.4f}" for loss in test_loss])
print("Final Score:", final_score)

# **시각화**

In [ ]:
# 시각화
plt.figure(figsize=(10, 5))

# 에러 값 그래프
plt.subplot(1, 2, 1)
plt.plot(train_error, label='Train Error')
plt.plot(test_error, label='Test Error')
plt.xlabel('Number of Iterations')
plt.ylabel('Error')
plt.title('Hyper-Prameter-Error')
plt.legend()

# 손실 값 그래프
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(test_loss, label='Test Loss')
plt.xlabel('Number of Iterations')
plt.ylabel('Loss')
plt.title('Hyper-Prameter-Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 24))
plot_importance(XGB_Best, ax=ax, max_num_features=8)